In [107]:
import sys
import pandas as pd

messages_df = pd.read_csv("../data/disaster_messages.csv")
categories_df = pd.read_csv("../data/disaster_categories.csv")
df = categories_df.merge(messages_df, left_on='id', right_on='id')


In [108]:
df

,id,categories,message,original,genre
0,2,related-1;request-0;offer-0;aid_related-0;medi...,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,related-1;request-0;offer-0;aid_related-1;medi...,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,related-1;request-0;offer-0;aid_related-0;medi...,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,related-1;request-1;offer-0;aid_related-1;medi...,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,related-1;request-0;offer-0;aid_related-0;medi...,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct
...,...,...,...,...,...
26381,30261,related-0;request-0;offer-0;aid_related-0;medi...,The training demonstrated how to enhance micro...,NaN,news
26382,30262,related-0;request-0;offer-0;aid_related-0;medi...,A suitable candidate has been selected and OCH...,NaN,news
26383,30263,related-1;request-0;offer-0;aid_related-0;medi...,"Proshika, operating in Cox's Bazar municipalit...",NaN,news
26384,30264,related-1;request-0;offer-0;aid_related-1;medi...,"Some 2,000 women protesting against the conduc...",NaN,news


In [109]:
#separate string value into individual column values by delimiter
categories = df['categories'].str.split(';',expand=True)

In [110]:
#pullout just column names
categories_colnames = categories.loc[0].str[:-2].tolist()


In [111]:
#change column names
categories.columns = categories_colnames


In [112]:
#turn values into 0 or 1 based on end of string value, then turn numeric
for columns in categories:
    categories[columns] = categories[columns].str[-1:].astype('int32')

In [113]:
#merge two data sets and drop "category" column
df.drop(columns="categories", inplace=True)


In [114]:
#combine and drop duplicate rows
df = pd.concat([df,categories], axis = 1)
df.drop_duplicates(keep='first', inplace=True)
df.reset_index()


,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,26381,30261,The training demonstrated how to enhance micro...,NaN,news,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,26382,30262,A suitable candidate has been selected and OCH...,NaN,news,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,26383,30263,"Proshika, operating in Cox's Bazar municipalit...",NaN,news,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,26384,30264,"Some 2,000 women protesting against the conduc...",NaN,news,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
#load data
from sqlalchemy import create_engine

engine = create_engine('sqlite:///disaster_records.db')
df.to_sql('disaster_messages', engine, index=False)
